## This code is functional on google collab. Otherwise, modify the paths

### Import library

In [1]:
!pip install -U ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 26.2 MB/s eta 0:00:00
   

### Load Dataset

In [2]:
!kaggle datasets download -d cryptexcode/mpst-movie-plot-synopses-with-tags -p /content/

Dataset URL: https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags
License(s): CC-BY-SA-4.0
 31% 9.00M/28.8M [00:00<00:00, 33.0MB/s]
100% 28.8M/28.8M [00:00<00:00, 78.9MB/s]


### Load the pretrained model

In [3]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


### Prepare the collection




In [4]:
import zipfile
import pandas as pd


with zipfile.ZipFile('/content/mpst-movie-plot-synopses-with-tags.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

df = pd.read_csv("/content/mpst_full_data.csv")

collection = df['plot_synopsis'][:100].tolist()
titles = df['title'][:100].tolist()


In [5]:
collection = [
    f"Title: {title}\nPlot: {synopsis}"
    for title, synopsis in zip(titles, collection)
]

In [6]:
print(len(df))

14828


## Create embeddings + Faiss Index which saves the embeddings

In [7]:
RAG.index(
    collection=collection,
    index_name="movie_plots_index",
    max_document_length=300,
    split_documents=False,
    bsize = 32,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Jan 12, 18:06:48] #> Creating directory .ragatouille/colbert/indexes/movie_plots_index 


[Jan 12, 18:06:50] [0] 		 #> Encoding 100 passages..


/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


[Jan 12, 18:06:52] [0] 		 avg_doclen_est = 257.5799865722656 	 len(local_sample) = 100
[Jan 12, 18:06:52] [0] 		 Creating 2,048 partitions.
[Jan 12, 18:06:52] [0] 		 *Estimated* 25,757 embeddings.
[Jan 12, 18:06:52] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/movie_plots_index/plan.json ..


/usr/local/lib/python3.10/dist-packages/colbert/indexing/collection_indexer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_sample = torch.load(sub_sample_path)


used 20 iterations (0.5347s) to cluster 24471 items into 2048 clusters
[Jan 12, 18:06:52] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


...

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
1it [00:01,  1.02s/it]
  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

[Jan 12, 18:09:55] #> Optimizing IVF to store map from centroids to list of pids..
[Jan 12, 18:09:55] #> Building the emb2pid mapping..
[Jan 12, 18:09:55] len(emb2pid) = 25758



100%|██████████| 2048/2048 [00:00<00:00, 61904.98it/s]

[Jan 12, 18:09:55] #> Saved optimized IVF to .ragatouille/colbert/indexes/movie_plots_index/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/movie_plots_index'

In [8]:
!mv /content/.ragatouille /content/movie_plots_index

### Load pretrained model

In [10]:
RAG = RAGPretrainedModel.from_index("/content/movie_plots_index/colbert/indexes/movie_plots_index")

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


### Searcher

In [11]:
results = RAG.search(query="A movie about space exploration and human survival", k=3)

Loading searcher for index movie_plots_index for the first time... This may take a few seconds
[Jan 12, 18:14:02] #> Loading codec...
[Jan 12, 18:14:02] #> Loading IVF...
[Jan 12, 18:14:02] #> Loading doclens...


/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids = torch.load(centroids_path, map_loc

[Jan 12, 18:14:02] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . A movie about space exploration and human survival, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 1037, 3185, 2055, 2686, 8993, 1998, 2529, 7691,  102,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')




/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


### Result

In [16]:
for i in range(3):
    print(f"Rank = {results[i]['score']}")
    print(f"{results[i]['content']}\n")

Rank = 14.3671875
Title: Night of the Lepus
Plot: The movie opens showing a news report about how overpopulation is not just a human problem, but exists in the animal kingdom as well, using the rabbit population explosion in Australia in the 1950s as an example. There was a similar plague of rabbits getting out of hand in the American Southwest, especially in Arizona, where the rabbits were devouring crops, destroying property, and out-competing livestock for a finite supply of food and water.Moving to the action in Arizona, our first view is of rancher Cole Hillman, riding his horse. The horse steps into a rabbit hole and injures itself, and Cole must shoot it and walk home. He instructs one of his ranch hands to take the truck to go get his saddle, then makes a phone call to the university in Ajo, hoping to find a cure for the rabbit infestation. He meets Dr. Elgin Clark, but his first suggestion is for the same person who killed all the coyotes that were keeping the rabbits in check

# Use as LangChain Retriever

In [13]:
retriever = RAG.as_langchain_retriever(k=3)

In [14]:
resultats = retriever.invoke("Movie that talk about animals")

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [15]:
for doc in resultats:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)

Content: Title: Night of the Lepus
Plot: The movie opens showing a news report about how overpopulation is not just a human problem, but exists in the animal kingdom as well, using the rabbit population explosion in Australia in the 1950s as an example. There was a similar plague of rabbits getting out of hand in the American Southwest, especially in Arizona, where the rabbits were devouring crops, destroying property, and out-competing livestock for a finite supply of food and water.Moving to the action in Arizona, our first view is of rancher Cole Hillman, riding his horse. The horse steps into a rabbit hole and injures itself, and Cole must shoot it and walk home. He instructs one of his ranch hands to take the truck to go get his saddle, then makes a phone call to the university in Ajo, hoping to find a cure for the rabbit infestation. He meets Dr. Elgin Clark, but his first suggestion is for the same person who killed all the coyotes that were keeping the rabbits in check. Cole is